In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def xywh2xyxy(xywh):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    x, y, w, h = xywh
    x1 = (x - w/2)*W
    y1 = (y - h/2)*H
    x2 = (x + w/2)*W
    y2 = (y + h/2)*H
    return [x1, y1, x2, y2]

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness
#     color = color or [random.randint(0, 255) for _ in range(3)]

    if label == 0: color = [0, 0, 255] # Đỏ
    elif label == 1: color = [255, 102, 0] # Xanh dương
    elif label == 2: color = [204, 0, 102] # Tím
    elif label == 3: color = [0, 153, 255] # Vàng cam
    elif label == 4: color = [0, 153, 0] # Xanh lá cây đậm
        
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    
    if label and 0:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

In [ ]:
%cd /root/AFS/SangDV-ScaledYOLOv4

from PIL import Image 
import glob, os, cv2
import numpy as np
import random, glob
from scipy import ndimage

import matplotlib.pyplot as plt
from PIL import Image

for f in glob.glob('./train/labels.cache'): os.remove(f)
for f in glob.glob('./train/images/*'): os.remove(f)
for f in glob.glob('./train/labels/*'): os.remove(f)

SOURCE = "./Scaled-Yolov4/ori_img/"
count_img = 0

noise1 = cv2.imread("./Scaled-Yolov4/noise1.png")
noise1 = cv2.cvtColor(noise1, cv2.COLOR_BGR2RGB)
noise2 = cv2.imread("./Scaled-Yolov4/noise2.png")
noise2 = cv2.cvtColor(noise2, cv2.COLOR_BGR2RGB)
noise3 = cv2.imread("./Scaled-Yolov4/noise3.png")
noise3 = cv2.cvtColor(noise3, cv2.COLOR_BGR2RGB)
ran_noise = random.randint(15, 50)
               
for img_file in os.listdir(SOURCE + "images"):
    if(img_file[:2]!= "f_"): continue
    img = np.array(Image.open(SOURCE + "images/" + img_file))
    H, W, C = img.shape
    if C==4: img = img[:,:,:3]
    print(SOURCE + "images/" + img_file, img.shape)
    
    for j in range(100*count_img, 100*(count_img + 1)):
        output_img = 255 * np.ones((H, W, 3), np.uint8)
        if(random.randint(0, 3) == 1): output_img[:] = (20,84,104)
        elif(random.randint(0, 3) == 2): output_img[:] = (30,119,148)
        elif(random.randint(0, 3) == 3): output_img[:] = (23,92,115)
        new_label_txt = open('./train/labels/'+str(j)+'.txt', "w")
        label_txt = open(SOURCE + "labels/"+ img_file[:-4] + ".txt")
        
    
        for line in label_txt:
            line = line.split()
            x,y,w,h=float(line[1]),float(line[2]),int(float(line[3])*W),int(float(line[4])*H)
            x1 = int(x*W - w/2)
            y1 = int(y*H - h/2)
            x2 = x1 + w
            y2 = y1 + h
            
            frequence = random.randint(2, 6)
            for i in range(frequence):
                # Toạ độ mới
                empty = 0
                count_failed = 0
                
                while empty == 0: 
                    count_failed += 1
                    if(count_failed > 100): break
                    crop = img[y1:y2, x1:x2, :]
                    crop_2 = ndimage.rotate(crop, angle=random.randint(0,180), cval=255)
                    x1_, y1_ = random.randint(0, W - crop_2.shape[1]), random.randint(0, H - crop_2.shape[0])
                    x2_ = x1_ + crop_2.shape[1]
                    y2_ = y1_ + crop_2.shape[0]
                    crop_2[crop_2.sum(axis=2) > 330] = (255,255,255)
                    # Nếu là giọt Drop thì cho phép paste luôn không cần xem xét là có đang đè không
                    empty = 1
#                     if(line[0] == '3'): empty = 1
#                     if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
                        
                if(empty == 1):
                    output_img[y1_:y2_, x1_:x2_, :][crop_2[:,:,:] < 180] = crop_2[crop_2[:,:,:] < 180]
                    
                    new_label_txt.write(line[0]+" " + str((x1_+x2_)/2/W) + " " + 
                        str((y1_+y2_)/2/H) + " " + str(crop_2.shape[1]/W) + " " + 
                        str(crop_2.shape[0]/H) + "\n")
        
        
        ############################################################################################
        ############################################################################################
        # Paste noise:
        for i in range(ran_noise):
            noise1_ = ndimage.rotate(noise1, angle=random.randint(0,180), cval=255)
            empty = 0
            count_failed = 0
            while empty == 0: 
                count_failed += 1
                if(count_failed > 20): break
                x1_, y1_ = random.randint(0, W - noise1_.shape[1]), random.randint(0, H - noise1_.shape[0])
                x2_ = x1_ + noise1_.shape[1]
                y2_ = y1_ + noise1_.shape[0]
                if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
            if(empty == 1):
                output_img[y1_:y2_, x1_:x2_, :][noise1_[:,:,0] < 255] = noise1_[noise1_[:,:,0] < 255]
            
            empty = 0
            count_failed = 0
            noise2_ = ndimage.rotate(noise2, angle=random.randint(0,180), cval=255)
            while empty == 0: 
                count_failed += 1
                if(count_failed > 20): break
                x1_, y1_ = random.randint(0, W - noise2_.shape[1]), random.randint(0, H - noise2_.shape[0])
                x2_ = x1_ + noise2_.shape[1]
                y2_ = y1_ + noise2_.shape[0]
                if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
            if(empty == 1):
                output_img[y1_:y2_, x1_:x2_, :][noise2_[:,:,0] < 255] = noise2_[noise2_[:,:,0] < 255]
            
            empty = 0
            count_failed = 0
            noise3_ = ndimage.rotate(noise3, angle=random.randint(0,180), cval=255)
            while empty == 0: 
                count_failed += 1
                if(count_failed > 20): break
                x1_, y1_ = random.randint(0, W - noise3_.shape[1]), random.randint(0, H - noise3_.shape[0])
                x2_ = x1_ + noise3_.shape[1]
                y2_ = y1_ + noise3_.shape[0]
                if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
            if(empty == 1):
                output_img[y1_:y2_, x1_:x2_, :][noise3_[:,:,0] < 255] = noise3_[noise3_[:,:,0] < 255]
        
#         print(max(np.unique(output_img)))
        Image.fromarray(output_img).save('./train/images/'+str(j)+'.png')   
#         mnmn = cv2.imread('./train/images/'+str(j)+'.png')
#         print(max(np.unique(output_img)))
        new_label_txt.close()
    
    count_img +=1
print("Create Traningset Done!")

/root/AFS/SangDV-ScaledYOLOv4
./Scaled-Yolov4/ori_img/images/f_01495.png (600, 1200, 3)
./Scaled-Yolov4/ori_img/images/f_01318.png (600, 1200, 3)
./Scaled-Yolov4/ori_img/images/f_01213-edit.png (600, 1200, 3)
./Scaled-Yolov4/ori_img/images/f_01477.png (1200, 1600, 3)


In [130]:
%cd /root/AFS/SangDV-ScaledYOLOv4
from PIL import Image 
import glob, os, cv2
import numpy as np
import random, glob

for f in glob.glob('./valid/labels.cache'): os.remove(f)
for f in glob.glob('./valid/images/*'): os.remove(f)
for f in glob.glob('./valid/labels/*'): os.remove(f)
    
SOURCE = "./Scaled-Yolov4/ori_img/"
count_img = 0

noise1 = cv2.imread("./Scaled-Yolov4/noise1.png")
noise1 = cv2.cvtColor(noise1, cv2.COLOR_BGR2RGB)
noise2 = cv2.imread("./Scaled-Yolov4/noise2.png")
noise2 = cv2.cvtColor(noise2, cv2.COLOR_BGR2RGB)
noise3 = cv2.imread("./Scaled-Yolov4/noise3.png")
noise3 = cv2.cvtColor(noise3, cv2.COLOR_BGR2RGB)
ran_noise = random.randint(15, 50)
               
for img_file in os.listdir(SOURCE + "images"):
    if(img_file[:2]!= "f_"): continue
    img = np.array(Image.open(SOURCE + "images/" + img_file))
    H, W, C = img.shape
    if C==4: img = img[:,:,:3]
    print(SOURCE + "images/" + img_file, img.shape)
    
    for j in range(20*count_img, 20*(count_img + 1)):
        output_img = 255 * np.ones((H, W, 3), np.uint8)
        if(random.randint(0, 3) == 1): output_img[:] = (20,84,104)
        elif(random.randint(0, 3) == 2): output_img[:] = (30,119,148)
        elif(random.randint(0, 3) == 3): output_img[:] = (23,92,115)
        new_label_txt = open('./valid/labels/'+str(j)+'.txt', "w")
        label_txt = open(SOURCE + "labels/"+ img_file[:-4] + ".txt")
        
        
        for line in label_txt:
            line = line.split()
            x,y,w,h=float(line[1]),float(line[2]),int(float(line[3])*W),int(float(line[4])*H)
            x1 = int(x*W - w/2)
            y1 = int(y*H - h/2)
            x2 = x1 + w
            y2 = y1 + h
            
            frequence = random.randint(2, 6)
            for i in range(frequence):
                # Toạ độ mới
                empty = 0
                count_failed = 0
                
                while empty == 0: 
                    count_failed += 1
                    if(count_failed > 100): break
                    crop = img[y1:y2, x1:x2, :]
                    crop_2 = ndimage.rotate(crop, angle=random.randint(0,180), cval=255)
                    x1_, y1_ = random.randint(0, W - crop_2.shape[1]), random.randint(0, H - crop_2.shape[0])
                    x2_ = x1_ + crop_2.shape[1]
                    y2_ = y1_ + crop_2.shape[0]
                    crop_2[crop_2.sum(axis=2) > 330] = (255,255,255)
                    # Nếu là giọt Drop thì cho phép paste luôn không cần xem xét là có đang đè không
                    empty = 1
#                     if(line[0] == '3'): empty = 1
#                     if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
                        
                if(empty == 1):
                    output_img[y1_:y2_, x1_:x2_, :][crop_2[:,:,:] < 200] = crop_2[crop_2[:,:,:] < 200]
                    
                    new_label_txt.write(line[0]+" " + str((x1_+x2_)/2/W) + " " + 
                        str((y1_+y2_)/2/H) + " " + str(crop_2.shape[1]/W) + " " + 
                        str(crop_2.shape[0]/H) + "\n")
        
        
        ############################################################################################
        ############################################################################################
        # Paste noise:
        for i in range(ran_noise):
            noise1_ = ndimage.rotate(noise1, angle=random.randint(0,180), cval=255)
            empty = 0
            count_failed = 0
            while empty == 0: 
                count_failed += 1
                if(count_failed > 20): break
                x1_, y1_ = random.randint(0, W - noise1_.shape[1]), random.randint(0, H - noise1_.shape[0])
                x2_ = x1_ + noise1_.shape[1]
                y2_ = y1_ + noise1_.shape[0]
                if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
            if(empty == 1):
                output_img[y1_:y2_, x1_:x2_, :][noise1_[:,:,0] < 255] = noise1_[noise1_[:,:,0] < 255]
            
            empty = 0
            count_failed = 0
            noise2_ = ndimage.rotate(noise2, angle=random.randint(0,180), cval=255)
            while empty == 0: 
                count_failed += 1
                if(count_failed > 20): break
                x1_, y1_ = random.randint(0, W - noise2_.shape[1]), random.randint(0, H - noise2_.shape[0])
                x2_ = x1_ + noise2_.shape[1]
                y2_ = y1_ + noise2_.shape[0]
                if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
            if(empty == 1):
                output_img[y1_:y2_, x1_:x2_, :][noise2_[:,:,0] < 255] = noise2_[noise2_[:,:,0] < 255]
            
            empty = 0
            count_failed = 0
            noise3_ = ndimage.rotate(noise3, angle=random.randint(0,180), cval=255)
            while empty == 0: 
                count_failed += 1
                if(count_failed > 20): break
                x1_, y1_ = random.randint(0, W - noise3_.shape[1]), random.randint(0, H - noise3_.shape[0])
                x2_ = x1_ + noise3_.shape[1]
                y2_ = y1_ + noise3_.shape[0]
                if (len(np.unique(output_img[y1_:y2_, x1_:x2_, :])) == 1): empty = 1
            if(empty == 1):
                output_img[y1_:y2_, x1_:x2_, :][noise3_[:,:,0] < 255] = noise3_[noise3_[:,:,0] < 255]
        
#         print(max(np.unique(output_img)))
        Image.fromarray(output_img).save('./valid/images/'+str(j)+'.png')   
#         mnmn = cv2.imread('./train/images/'+str(j)+'.png')
#         print(max(np.unique(output_img)))
        new_label_txt.close()
    
    count_img +=1
print("Create Validation Done!")

/root/AFS/SangDV-ScaledYOLOv4
./Scaled-Yolov4/ori_img/images/f_01495.png (600, 1200, 3)
./Scaled-Yolov4/ori_img/images/f_01318.png (600, 1200, 3)
./Scaled-Yolov4/ori_img/images/f_01213-edit.png (600, 1200, 3)
./Scaled-Yolov4/ori_img/images/f_01477.png (1200, 1600, 3)
./Scaled-Yolov4/ori_img/images/f_01477-edit.png (1200, 1600, 3)
./Scaled-Yolov4/ori_img/images/f_01554.png (1200, 1600, 3)
./Scaled-Yolov4/ori_img/images/f_01554-edit.png (1200, 1600, 3)
./Scaled-Yolov4/ori_img/images/f_01100.png (1200, 1600, 3)
Create Validation Done!


In [ ]:
!pip install torch==1.7.1 torchvision==0.8.2
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U PyYAML

# # clone Scaled_YOLOv4
# !git clone https://github.com/WongKinYiu/ScaledYOLOv4.git  # clone repo
# %cd ScaledYOLOv4/
# #checkout the yolov4-large branch
# !git checkout yolov4-large

# #install mish activation funciton for cuda
# %cd ~/AFS/Plate_Scaled_Yolov4/
# !git clone https://github.com/JunnYu/mish-cuda
# %cd mish-cuda
# !python setup.py build install

In [ ]:
%cd Scaled-Yolov4
!python train.py --img 1024 --batch 8 --epochs 5000 --data '../data.yaml' --cfg ./models/yolov4-csp.yaml --weights '' --name yolov4-csp-results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!python detect.py --weights ./runs/exp21_yolov4-csp-results/weights/best_yolov4-csp-results.pt --img 1024 --conf 0.09 --source ../test/images